# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import cartopy.crs as ccrs

from geoviews import annotate
gv.extension('bokeh')
# Import API key
from api_keys import geoapi_key


C:\Users\micha\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,saipan,15.1355,145.7010,298.54,78,20,6.17,MP,1707501552
1,cabedelo,-6.9811,-34.8339,305.52,71,37,5.40,BR,1707501553
2,coquimbo,-29.9533,-71.3436,295.00,78,20,6.17,CL,1707501397
3,polyarnyy,69.1989,33.4478,255.50,96,100,2.46,RU,1707501553
4,amahai,-3.3333,128.9167,299.23,86,72,2.84,ID,1707501554


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humid_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City"
)

# Display the map
humid_map 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
target_cities = city_data_df.loc[(city_data_df["Max Temp"] >= 290) & 
                                 (city_data_df["Max Temp"] <=310) & 
                                 (city_data_df["Humidity"] <30) &
                                 (city_data_df["Wind Speed"] < 5)
                                ]
target_cities = target_cities.reset_index(drop=True)

# Drop any rows with null values
target_cities = target_cities.dropna()

# Display sample data
target_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bassila,9.0081,1.6654,303.90,13,10,0.87,BJ,1707501565
1,el bauga,18.2620,33.9081,296.46,17,5,4.92,SD,1707501576
2,taoudenni,22.6783,-3.9836,298.13,7,52,2.36,ML,1707501660
3,kyabe,9.4515,18.9449,304.12,15,60,1.87,TD,1707501687
4,kidal,18.4411,1.4078,298.52,7,85,1.89,ML,1707501723
5,chimbarongo,-34.7000,-71.0500,301.64,19,0,3.62,CL,1707501747
6,orapa,-21.2833,25.3667,304.61,25,6,1.83,BW,1707501752
7,koton-karfe,8.1000,6.8000,305.71,20,13,1.41,NG,1707501755
8,bouna,9.2667,-3.0000,306.00,9,24,2.46,CI,1707501759
9,boda,4.3189,17.4695,302.70,21,38,2.16,CF,1707501761


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = target_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bassila,BJ,9.0081,1.6654,13,
1,el bauga,SD,18.2620,33.9081,17,
2,taoudenni,ML,22.6783,-3.9836,7,
3,kyabe,TD,9.4515,18.9449,15,
4,kidal,ML,18.4411,1.4078,7,
5,chimbarongo,CL,-34.7000,-71.0500,19,
6,orapa,BW,-21.2833,25.3667,25,
7,koton-karfe,NG,8.1000,6.8000,20,
8,bouna,CI,9.2667,-3.0000,9,
9,boda,CF,4.3189,17.4695,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 15_000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey":geoapi_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bassila - nearest hotel: No hotel found
el bauga - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
kyabe - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
chimbarongo - nearest hotel: Hotel Central
orapa - nearest hotel: Makumutu Lodge
koton-karfe - nearest hotel: No hotel found
bouna - nearest hotel: Hôtel Éléphant  Bouna
boda - nearest hotel: No hotel found
el obeid - nearest hotel: فندق المدينة
goz beida - nearest hotel: No hotel found
okuta - nearest hotel: No hotel found
i-n-salah - nearest hotel: باجودة
gao - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
0,bassila,BJ,9.0081,1.6654,13,No hotel found
1,el bauga,SD,18.2620,33.9081,17,No hotel found
2,taoudenni,ML,22.6783,-3.9836,7,No hotel found
3,kyabe,TD,9.4515,18.9449,15,No hotel found
4,kidal,ML,18.4411,1.4078,7,No hotel found
5,chimbarongo,CL,-34.7000,-71.0500,19,Hotel Central
6,orapa,BW,-21.2833,25.3667,25,Makumutu Lodge
7,koton-karfe,NG,8.1000,6.8000,20,No hotel found
8,bouna,CI,9.2667,-3.0000,9,Hôtel Éléphant Bouna
9,boda,CF,4.3189,17.4695,21,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

map_plot = hotel_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    colormap='viridis',
    size='Humidity',
    title='City Humidity with Hotel Information',
    xlabel='Longitude',
    ylabel='Latitude',
    colorbar=True,
    frame_width=700,
    frame_height=500,
    clim=(0, 100),  # Set the colorbar range from 0 to 100
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name']  # Add 'Hotel Name' and 'Country' to hover information
)

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)